In [70]:
from datasets import load_dataset

dataset = load_dataset("../data/datasets/summary_title") # TODO: create real dataset

dataset

Using custom data configuration summary_title-d54ff16b9a7a2331


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/Marvin Kosmider/.cache/huggingface/datasets/csv/summary_title-d54ff16b9a7a2331/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text'],
        num_rows: 6
    })
})

In [71]:
def add_prefix(examples):
    examples["text"] = "Write a title: " + examples["text"]
    return examples

prefixed_dataset = dataset.map(add_prefix)
prefixed_dataset["train"][0]

  0%|          | 0/6 [00:00<?, ?ex/s]

{'id': 1,
 'title': 'I did a thing!',
 'text': 'Write a title: Tom set out to do a thing.'}

In [72]:
split_datasets = prefixed_dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text'],
        num_rows: 5
    })
    test: Dataset({
        features: ['id', 'title', 'text'],
        num_rows: 1
    })
})

In [73]:
split_datasets["validation"] = split_datasets.pop("test")

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B", return_tensors="pt")

https://huggingface.co/course/chapter7/4?fw=pt

In [78]:
def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["title"]
    model_inputs = tokenizer(
        inputs, text_target=targets, padding="max_length", truncation=True
    )
    return model_inputs

In [79]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [80]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")

In [81]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [83]:
import evaluate

metric = evaluate.load("sacrebleu")

import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [85]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"t0-3-tom-scott",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

In [86]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [87]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1
  Batch size = 64
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 42.594970703125,
 'eval_bleu': 13.134549472120788,
 'eval_runtime': 98.7375,
 'eval_samples_per_second': 0.01,
 'eval_steps_per_second': 0.01}

In [88]:
trainer.train()

C:\Users\Marvin Kosmider\FH Technikum Wien\Studium\Semester 2\Development Project 1\scientificProject\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 167772160 bytes.